# 🔐 ML Security Workshop: Serverless

[![Colab Badge](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/unionai-oss/ml-security/blob/main/workshop_serverless.ipynb)

In [11]:
%pip install -U 'flytekit>=0.14.0' union joblib openai pandas pyarrow scikit-learn

  Using cached flytekit-1.14.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached flyteidl-1.14.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached mashumaro-3.15-py3-none-any.whl.metadata (117 kB)
  Using cached msgpack-1.1.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (8.4 kB)
Using cached flytekit-1.14.0-py3-none-any.whl (620 kB)
Using cached flyteidl-1.14.0-py3-none-any.whl (218 kB)
Using cached mashumaro-3.15-py3-none-any.whl (93 kB)
Using cached msgpack-1.1.0-cp311-cp311-macosx_11_0_arm64.whl (81 kB)
  Attempting uninstall: mashumaro
    Found existing installation: mashumaro 3.13.1
    Uninstalling mashumaro-3.13.1:
      Successfully uninstalled mashumaro-3.13.1
  Attempting uninstall: flyteidl
    Found existing installation: flyteidl 1.13.4
    Uninstalling flyteidl-1.13.4:
      Successfully uninstalled flyteidl-1.13.4
  Attempting uninstall: flytekit
    Found existing installation: flytekit 1.13.9
    Uninstalling flytekit-1.13.9:
      Successfully uninstalled flytekit-1.13

## Login to Union Serverless

In [1]:
!union create login --auth device-flow --serverless

Login successful into serverless


## Training workflow

In [20]:
from functools import partial
from typing import NamedTuple

import joblib
import pandas as pd

import union
from flytekit.deck import MarkdownRenderer
from flytekit.types.file import FlyteFile

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score



image = union.ImageSpec(
    name="ml-security",
    requirements="requirements.txt",
)

task = partial(
    union.task,
    container_image=image,
    cache=True,
    cache_version="3",
)

ModelOutput = NamedTuple("Output", [("model", FlyteFile), ("accuracy", float)])


@task
def load_data() -> tuple[pd.DataFrame, pd.Series]:
    wine = load_wine()
    X = pd.DataFrame(wine.data, columns=wine.feature_names)
    y = pd.Series(wine.target)
    return X, y


@task
def split_data(X: pd.DataFrame, y: pd.Series) -> tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    return train_test_split(X, y, test_size=0.2, random_state=42)


@task
def train_model(X_train: pd.DataFrame, y_train: pd.Series) -> FlyteFile:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    path = "model.joblib"
    joblib.dump(model, path)
    return FlyteFile(path=path)


@task(enable_deck=True)
def evaluate_model(model: FlyteFile, X_test: pd.DataFrame, y_test: pd.Series) -> float:
    with open(model, "rb") as f:
        model = joblib.load(f)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    deck = union.Deck(name="Accuracy Report", html=MarkdownRenderer().to_html(f"# Test accuracy: {accuracy}"))
    union.current_context().decks.insert(0, deck)
    return accuracy


@union.workflow
def wine_classification_workflow() -> ModelOutput:
    X, y = load_data()
    X_train, X_test, y_train, y_test = split_data(X, y)
    model = train_model(X_train, y_train)
    accuracy = evaluate_model(model, X_test, y_test)
    return model, accuracy

In [21]:
from union.remote import UnionRemote

serverless = UnionRemote()

10:03:04.399845 WARNING  remote.py:286 - Jupyter notebook and interactive task  
                         support is still alpha.

In [22]:
execution = serverless.execute(wine_classification_workflow, inputs={})
execution

Flyte Serialized object (FlyteWorkflowExecution):
  id:
    project: flytesnacks
    domain: development
    name: ags6nzcxvdg56djhtstp
  spec:
    launch_plan:
      resource_type: 3
      project: flytesnacks
      domain: development
      name: wine_classification_workflow
      version: lZ-u_0gOKvlV2-3OIVaySg
    metadata:
      principal: 00ug8tl4tit8Dvpi65d7
  closure:
    created_at:
      seconds: 1733929386
      nanos: 800177000
    updated_at:
      seconds: 1733929386
      nanos: 800177000

In [23]:
execution.wait(poll_interval=1)
model_file = execution.outputs["model"]

with open(model_file, "rb") as f:
    model = joblib.load(f)

model

RandomForestClassifier(random_state=42)

In [24]:
features, _ = load_data()
predictions = model.predict(features)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

## Serving the model in batch mode

In [27]:
Predictions = NamedTuple("Predictions", [("predictions", list[float])])

@union.task(container_image=image)
def batch_predict(model: FlyteFile, data: pd.DataFrame) -> Predictions:
    with open(model, "rb") as f:
        model = joblib.load(f)
    return Predictions([float(x) for x in model.predict(data)])

In [28]:
execution = serverless.execute(
    batch_predict,
    inputs={"model": model_file, "data": features}
)
execution

Flyte Serialized object (FlyteWorkflowExecution):
  id:
    project: flytesnacks
    domain: development
    name: acfq6tktrp2s7sngbndj
  spec:
    launch_plan:
      resource_type: 1
      project: flytesnacks
      domain: development
      name: batch_predict
      version: FRQCWA41A1sKDOP6TaxIVA
    metadata:
      principal: 00ug8tl4tit8Dvpi65d7
  closure:
    created_at:
      seconds: 1733930132
      nanos: 514899000
    updated_at:
      seconds: 1733930132
      nanos: 514899000

In [30]:
execution.wait(poll_interval=1)
predictions = execution.outputs["predictions"]
predictions[:5]

[0.0, 0.0, 0.0, 0.0, 0.0]

## 🥒 The Pickle Attack

In [38]:
class PickleAttack:
    def __init__(self): ...

    def __reduce__(self):
        # os.system will execute the command
        import os
        return (os.system, ('echo "👋 Hello there, I\'m a pickle attack! 🥒"',))


fake_model = PickleAttack()
fake_model_path ="./fake_model.joblib"
with open(fake_model_path, "wb") as f:
    joblib.dump(fake_model, f)

fake_model_path

'./fake_model.joblib'

In [39]:
execution = serverless.execute(
    batch_predict, inputs={"model": fake_model_path, "data": features}
)
execution

Flyte Serialized object (FlyteWorkflowExecution):
  id:
    project: flytesnacks
    domain: development
    name: a9wfzphqlz6lpfnmnhmg
  spec:
    launch_plan:
      resource_type: 1
      project: flytesnacks
      domain: development
      name: batch_predict
      version: FRQCWA41A1sKDOP6TaxIVA
    metadata:
      principal: 00ug8tl4tit8Dvpi65d7
  closure:
    created_at:
      seconds: 1733932006
      nanos: 776541000
    updated_at:
      seconds: 1733932006
      nanos: 776541000

## Mitigation: include md5hash metadata

In [44]:
import hashlib
from dataclasses import dataclass


@dataclass
class Model:
    file: FlyteFile
    md5hash: str

    def __post_init__(self):
        with open(self.file, "rb") as f:
            md5hash = hashlib.md5(f.read()).hexdigest()
        if md5hash != self.md5hash:
            raise ValueError(
                "⛔️ Model md5hash mismatch: expected "
                f"{self.md5hash}, found {md5hash}."
            )
        
ModelOutput = NamedTuple("Output", [("model", Model), ("accuracy", float)])

@task
def secure_train_model(X_train: pd.DataFrame, y_train: pd.Series) -> Model:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    path = "model.joblib"
    joblib.dump(model, path)
    md5hash = hashlib.md5(open(path, 'rb').read()).hexdigest()
    return Model(file=FlyteFile(path=path), md5hash=md5hash)


@task(enable_deck=True)
def secure_evaluate_model(model: Model, X_test: pd.DataFrame, y_test: pd.Series) -> float:
    with open(model.file, "rb") as f:
        model = joblib.load(f)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    deck = union.Deck(name="Accuracy Report", html=MarkdownRenderer().to_html(f"# Test accuracy: {accuracy}"))
    union.current_context().decks.insert(0, deck)
    return accuracy


@union.workflow
def secure_wine_classification_workflow() -> ModelOutput:
    X, y = load_data()
    X_train, X_test, y_train, y_test = split_data(X, y)
    model = secure_train_model(X_train, y_train)
    accuracy = secure_evaluate_model(model, X_test, y_test)
    return model, accuracy

In [45]:
execution = serverless.execute(secure_wine_classification_workflow, inputs={})
execution

Flyte Serialized object (FlyteWorkflowExecution):
  id:
    project: flytesnacks
    domain: development
    name: apbvl6f2bbw6l4p5sb6r
  spec:
    launch_plan:
      resource_type: 3
      project: flytesnacks
      domain: development
      name: secure_wine_classification_workflow
      version: pDE2BV9J9Spm9ymXyitoiw
    metadata:
      principal: 00ug8tl4tit8Dvpi65d7
  closure:
    created_at:
      seconds: 1733932131
      nanos: 105835000
    updated_at:
      seconds: 1733932131
      nanos: 105835000

In [50]:
execution.wait(poll_interval=1)
model_file = execution.outputs["model"]

with open(model_file.file, "rb") as f:
    model = joblib.load(f)

print(f"md5hash: {model_file.md5hash}")
model

md5hash: 9ccceac3887204138d267fadc947e367


RandomForestClassifier(random_state=42)

In [59]:
@union.task(container_image=image)
def model_guard(model: FlyteFile, md5hash: str) -> Model:
    return Model(file=model, md5hash=md5hash)


@union.task(container_image=image)
def secure_batch_predict(model: Model, data: pd.DataFrame) -> Predictions:
    with open(model.file, "rb") as f:
        model = joblib.load(f)
    return Predictions([float(x) for x in model.predict(data)])


@union.workflow
def secure_batch_prediction_workflow(
    model: FlyteFile,
    md5hash: str,
    data: pd.DataFrame
) -> Predictions:
    checked_model = model_guard(model, md5hash)
    return secure_batch_predict(checked_model, data)

Generate predictions with the correct model

In [60]:
execution = serverless.execute(
    secure_batch_prediction_workflow,
    inputs={
        "model": model_file.file,
        "md5hash": model_file.md5hash,
        "data": features
    }
)
execution

Flyte Serialized object (FlyteWorkflowExecution):
  id:
    project: flytesnacks
    domain: development
    name: adrht7788hr9cfjpnng9
  spec:
    launch_plan:
      resource_type: 3
      project: flytesnacks
      domain: development
      name: secure_batch_prediction_workflow
      version: elPvFnFGeOdrSSx7J2j9ag
    metadata:
      principal: 00ug8tl4tit8Dvpi65d7
  closure:
    created_at:
      seconds: 1733932639
      nanos: 577619000
    updated_at:
      seconds: 1733932639
      nanos: 577619000

In [61]:
execution.wait(poll_interval=1)
predictions = execution.outputs["predictions"]
predictions[:5]

[0.0, 0.0, 0.0, 0.0, 0.0]

Call the secure batch prediction workflow with the fake model

In [62]:
execution = serverless.execute(
    secure_batch_prediction_workflow,
    inputs={
        "model": fake_model_path,
        "md5hash": model_file.md5hash,
        "data": features
    }
)
execution

Flyte Serialized object (FlyteWorkflowExecution):
  id:
    project: flytesnacks
    domain: development
    name: afps4p8mg4rxp8pb5db5
  spec:
    launch_plan:
      resource_type: 3
      project: flytesnacks
      domain: development
      name: secure_batch_prediction_workflow
      version: elPvFnFGeOdrSSx7J2j9ag
    metadata:
      principal: 00ug8tl4tit8Dvpi65d7
  closure:
    created_at:
      seconds: 1733933291
      nanos: 446490000
    updated_at:
      seconds: 1733933291
      nanos: 446490000